# Import

In [1]:
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
import numpy as np
import copy

from kesacco import clustpipe

# Parameters

### Define the simulation

In [2]:
cpipe = clustpipe.ClusterPipe(silent=False, output_dir='/Users/adam/Project/CTA/Phys/Outputs/KESACCO_example')

             _  __  ___    __    __     ___   ___   __         
            | |/ / | __| /' _/  /  \   / _/  / _/  /__\        
            |   <  | _|  `._`. | /\ | | \__ | \__ | \/ |       
            |_|\_\ |___| |___/ |_||_|  \__/  \__/  \__/        
Keen Event Simulation and Analysis for CTA Cluster Observations
---------------------------------------------------------------


### Define the cluster object

In [3]:
# Set the cluster basic properties
cpipe.cluster.name     = 'Perseus'
cpipe.cluster.redshift = 0.0179
cpipe.cluster.M500     = 6.2e14*u.solMass # Used for getting R500 and the pressure profile
cpipe.cluster.coord    = SkyCoord("3h19m47.2s +41d30m47s", frame='icrs')

# Truncate the cluster at 1.5 deg to allow small map field of view (for faster code) without loss of flux
cpipe.cluster.theta_truncation = 1.5*u.deg

# The target gas density [Churazov et al. 2003]
cpipe.cluster.density_gas_model = {'name':'doublebeta', 'beta1':1.8, 'r_c1':80*u.kpc, 'n_01':3.9e-2*u.cm**-3,
                                   'beta2':0.87, 'r_c2':280*u.kpc, 'n_02':4.05e-3*u.cm**-3}

# The thermal profile (assuming Planck 2013 UPP)
cpipe.cluster.set_pressure_gas_gNFW_param('P13UPP')

# CR physics
cpipe.cluster.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.2}
cpipe.cluster.set_density_crp_isodens_scal_param(scal=1.0)    # Assumes that CR follow thermal gas density
cpipe.cluster.X_crp_E = {'X':1.0, 'R_norm':cpipe.cluster.R500} # Assumes large amount of CR (10% instead of ~1% of E_th)

# Sampling
cpipe.cluster.map_reso = 0.01*u.deg      # Ideally should be few times smaller than the PSF
cpipe.cluster.Npt_per_decade_integ = 30

# Get information about the state of the cluster model
cpipe.cluster.print_param()

--- silent
    True
    <class 'bool'>
--- output_dir
    /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example
    <class 'str'>
--- cosmo
    FlatLambdaCDM(name="Planck15", H0=67.7 km / (Mpc s), Om0=0.307, Tcmb0=2.725 K, Neff=3.05, m_nu=[0.   0.   0.06] eV, Ob0=0.0486)
    <class 'astropy.cosmology.core.FlatLambdaCDM'>
--- name
    Perseus
    <class 'str'>
--- coord
    <SkyCoord (ICRS): (ra, dec) in deg
    (49.94666667, 41.51305556)>
    <class 'astropy.coordinates.sky_coordinate.SkyCoord'>
--- redshift
    0.0179
    <class 'float'>
--- D_ang
    77.50175857119436 Mpc
    <class 'astropy.units.quantity.Quantity'>
--- D_lum
    80.30115386650692 Mpc
    <class 'astropy.units.quantity.Quantity'>
--- M500
    620000000000000.0 solMass
    <class 'astropy.units.quantity.Quantity'>
--- R500
    1317.2929549772916 kpc
    <class 'astropy.units.quantity.Quantity'>
--- theta500
    0.9738530853229961 deg
    <class 'astropy.units.quantity.Quantity'>
--- R_truncation
    2.0289912947296167

### Define the point source object

In [4]:
# source 1
name='NGC1275'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h19m48.16s +41d30m42s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h19m48.16s +41d30m42s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':2.1e-11/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-3.6, 'free':True},
                   'PivotEnergy':{'value':0.2*u.TeV, 'free':False}}}

#cpipe.compact_source.add_source(name, spatial, spectral) # Add the source to the model

In [5]:
# source 2
name='IC310'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h16m42.98s +41d19m30s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h16m42.98s +41d19m30s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':4.3e-12/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-1.95, 'free':True},
                   'PivotEnergy':{'value':1.0*u.TeV, 'free':False}}}

cpipe.compact_source.add_source(name, spatial, spectral)# Add the source to the model

In [6]:
# Show the status of the compact sources in the sky model
cpipe.compact_source.print_source()

--- IC310
    -- Spatial model: PointSource
         RA: {'value': <Longitude 49.17908333 deg>, 'free': False}
         DEC: {'value': <Latitude 41.325 deg>, 'free': False}
    -- Spectral model: PowerLaw
         Prefactor: {'value': <Quantity 4.3e-12 1 / (cm2 s TeV)>, 'free': True}
         Index: {'value': -1.95, 'free': True}
         PivotEnergy: {'value': <Quantity 1. TeV>, 'free': False}
    -- Temporal model: Constant
         Normalization: {'value': 1.0, 'free': False}


### Define the observations

In [7]:
# Assumes 3 hours (3 x 1 hours) with pointings 1 deg offset from the cluster in a triangle shape
# Use standard IRF

# One pointing offset +0 +1
cpipe.obs_setup.add_obs(obsid='001', name='Perseus_Ptg1', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+0, cpipe.cluster.coord.dec.value+1, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-01T00:00:00.0', tmax='2020-01-01T00:01:00.0', deadc=0.95)

# One pointing offset +sqrt(3)/2 -0.5
cpipe.obs_setup.add_obs(obsid='002', name='Perseus_Ptg2', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+np.sqrt(3)/2, cpipe.cluster.coord.dec.value-0.5, unit='deg'),
                        rad=5*u.deg, 
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-02T00:00:00.0', tmax='2020-01-02T00:01:00.0', deadc=0.95)

# One pointing offset -sqrt(3)/2 -0.5
cpipe.obs_setup.add_obs(obsid='003', name='Perseus_Ptg3', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value-np.sqrt(3)/2, cpipe.cluster.coord.dec.value-0.5, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-03T00:00:00.0', tmax='2020-01-03T00:01:00.0', deadc=0.95)

# Print info
cpipe.obs_setup.print_obs()

=== Perseus_Ptg1, ObsID 001
        RA-Dec:    49.94666666666666, 42.51305555555555 deg
        GLON-GLAT: 150.0078784158764, -12.426877669678154 deg
        ROI rad: 5.0 deg
        tmin: 2020-01-01T00:00:00.0
        tmax: 2020-01-01T00:01:00.0
        emin: 0.05 TeV
        emax: 100.0 TeV
        deadc: 0.95
        caldb: prod3b-v2
        irf: North_z20_S_5h
        bkg: name Background, obsid None, spatial type CTAIrfBackground, spectral type PowerLaw
=== Perseus_Ptg2, ObsID 002
        RA-Dec:    50.8126920704511, 41.01305555555555 deg
        GLON-GLAT: 151.41415408419417, -13.314435526241189 deg
        ROI rad: 5.0 deg
        tmin: 2020-01-02T00:00:00.0
        tmax: 2020-01-02T00:01:00.0
        emin: 0.05 TeV
        emax: 100.0 TeV
        deadc: 0.95
        caldb: prod3b-v2
        irf: North_z20_S_5h
        bkg: name Background, obsid None, spatial type CTAIrfBackground, spectral type PowerLaw
=== Perseus_Ptg3, ObsID 003
        RA-Dec:    49.08064126288222, 41.01305

# Run the simulation

In [8]:
cpipe.run_sim_obs()
cpipe.run_sim_quicklook(ShowEvent=True,
                        ShowObsDef=True,
                        ShowSkyModel=True,
                        bkgsubtract='NONE',
                        smoothing_FWHM=0.2*u.deg)

----- ObsID to be observed: ['001', '002', '003']
------- Simulation log -------
=== GApplication ===
 Name ......................: ctobssim
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Sim_ObsDef.xml
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Sim_Model_Unstack.xml
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 edisp .....................: no
 outevents .................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Events.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/TmpEvents
 startindex ................: 1
 seed ......................: 832456
 ra ........................: 50.9466666666667
 dec .......................: 42.5130555555556
 rad .......................: 8
 tmin ......................: 2020-02-01T00:00:00.0
 tmax ......................: 2020-02-01T00:20:00.0
 mjdref ........

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting_irf.py:111: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  c    = sub.imshow(image, extent=[emin,emax,tmin,tmax], aspect=0.5, vmin=1e6, vmax=5e10, norm=LogNorm())
/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting_irf.py:473: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  c    = sub.imshow(image, extent=[emin,emax,tmin,tmax], aspect=0.5,


!!! WARNING: XSPEC_table.txt not generated, skip Xray flux and Sx
!!! WARNING: XSPEC_table.txt not generated, skip Xray spectrum
!!! WARNING: XSPEC_table.txt not generated, skip PLOT_MAP_X
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Events001.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Sim_Skymap001.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 501
 nypix .....................: 501
 binsz .....................: 0.01
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:382: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam/Project/anaconda3/envs/cta_phys/lib/python3.8/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)


----- Quicklook 1/3 done
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Events002.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Sim_Skymap002.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 501
 nypix .....................: 501
 binsz .....................: 0.01
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.1
 inradius ..................: 1
 outradius .................: 2
 iterations ................: 3
 threshold ................

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:382: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam/Project/anaconda3/envs/cta_phys/lib/python3.8/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)


----- Quicklook 2/3 done
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Events003.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Sim_Skymap003.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 501
 nypix .....................: 501
 binsz .....................: 0.01
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.1
 inradius ..................: 1
 outradius .................: 2
 iterations ................: 3
 threshold ................

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:382: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam/Project/anaconda3/envs/cta_phys/lib/python3.8/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)


----- Quicklook 3/3 done


# Run the analysis

In [9]:
#----- Analysis parameters
cpipe.method_binned = True   # Do a binned analysis
cpipe.method_stack  = True   # Stack the event from different observations in a single analysis?
cpipe.spec_enumbins = 10
cpipe.spec_emin     = 50*u.GeV
cpipe.spec_emax     = 100*u.TeV

# Force the use of the user defined map grid
cpipe.map_UsePtgRef     = False
# Define the map used for the binned analysis 
cpipe.map_reso          = 0.1*u.deg # Can be increaded if the code is too slow 
cpipe.map_fov           = 3*u.deg    # Can also be reduced (but should increase bkg-cluster degeneracy)
cpipe.map_coord         = copy.deepcopy(cpipe.cluster.coord)
# Define the map used for the template
cpipe.cluster.map_fov   = 2.1*cpipe.cluster.theta_truncation
cpipe.cluster.map_coord = copy.deepcopy(cpipe.cluster.coord)

In [9]:
#----- Run the analysis
cpipe.run_analysis(refit=False, 
                   like_accuracy=0.05,
                   max_iter=50,
                   fix_spat_for_ts=False,
                   imaging_bkgsubtract='NONE',
                   do_Skymap=True, 
                   do_SourceDet=True, 
                   do_ResMap=True, 
                   do_TSmap=False,           # Warning, TS maps are very long to compute
                   profile_reso=0.05*u.deg,
                   do_Spec=True, 
                   do_Butterfly=True, 
                   do_Resspec=True,
                   smoothing_FWHM=0.1*u.deg, 
                   profile_log=True)

----- ObsID to be analysed: ['001', '002', '003']

=== GApplication ===
 Name ......................: ctselect
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Events.xml
 outobs ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_EventsSelected.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Selected
 usepnt ....................: no
 ra ........................: 49.9466666666667
 dec .......................: 41.5130555555556
 rad .......................: 2.12132034355964
 forcesel ..................: yes
 tmin ......................: NONE
 tmax ......................: NONE
 emin ......................: 0.05
 emax ......................: 100
 phase .....................: NONE
 expr ......................: 
 usethres ..................: NONE
 publish ...................: no
 chatter ...................: 2
 clobber ...................: yes
 debug

=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_EventsSelected.xml
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_SkymapTot.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 151
 nypix .....................: 151
 binsz .....................: 0.02
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.1
 inradius ..................: 0.6
 outradius .................: 0.8
 iterations ................: 3
 threshold .................: 3
 inexclu

--- Computing spectrum: Perseus
=== GApplication ===
 Name ......................: csspec
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Output.xml
 srcname ...................: Perseus
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Bkgcube.fits
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outfile ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Spectrum_Perseus.fits
 method ....................: SLICE
 ebinalg ...................: LOG
 emin

=== GApplication ===
 Name ......................: csresspec
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Output.xml
 modcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Cube.fits
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Bkgcube.fits
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outfile ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Spectrum_Residual.fits
 statistic .................: DEFAULT


/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting_irf.py:111: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  c    = sub.imshow(image, extent=[emin,emax,tmin,tmax], aspect=0.5, vmin=1e6, vmax=5e10, norm=LogNorm())
/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting_irf.py:473: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  c    = sub.imshow(image, extent=[emin,emax,tmin,tmax], aspect=0.5,


...

=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_SelectedEvents002.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Skymap002.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 251
 nypix .....................: 251
 binsz .....................: 0.02
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.1
 inradius ..................: 1
 outradius .................: 2
 iterations ................: 3
 threshold .................: 3
 inexclu

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:382: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam/Project/anaconda3/envs/cta_phys/lib/python3.8/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)


=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_SelectedEvents003.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Skymap003.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 251
 nypix .....................: 251
 binsz .....................: 0.02
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.1
 inradius ..................: 1
 outradius .................: 2
 iterations ................: 3
 threshold .................: 3
 inexclu

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:382: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam/Project/anaconda3/envs/cta_phys/lib/python3.8/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)
/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:382: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam

         This assumes weak noise spatial variarion (w.r.t. smoothing),
         gaussian regime, and uncorrelated pixels.
         This assumes weak noise spatial variarion (w.r.t. smoothing),
         gaussian regime, and uncorrelated pixels.


In [11]:
#cpipe.run_ana_dataprep(obsID=obsID)

----- ObsID to be analysed: ['001', '002', '003']

=== GApplication ===
 Name ......................: ctselect
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Events.xml
 outobs ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_EventsSelected.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Selected
 usepnt ....................: no
 ra ........................: 49.9466666666667
 dec .......................: 41.5130555555556
 rad .......................: 2.12132034355964
 forcesel ..................: yes
 tmin ......................: NONE
 tmax ......................: NONE
 emin ......................: 0.05
 emax ......................: 100
 phase .....................: NONE
 expr ......................: 
 usethres ..................: NONE
 publish ...................: no
 chatter ...................: 2
 clobber ...................: yes
 debug

In [12]:
#cpipe.run_ana_likelihood(refit=False, like_accuracy=0.05, max_iter=50, fix_spat_for_ts=False)

=== GOptimizerLM ===
 Optimized function value ..: 2826.730
 Absolute precision ........: 0.05
 Acceptable value decrease .: 2
 Optimization status .......: converged
 Number of parameters ......: 13
 Number of free parameters .: 5
 Number of iterations ......: 7
 Lambda ....................: 1e-08
=== GObservations ===
 Number of observations ....: 1
 Number of models ..........: 3
 Number of observed events .: 3361
 Number of predicted events : 3360.99840423672
=== GModels ===
 Number of models ..........: 3
 Number of parameters ......: 13
=== GModelSky ===
 Name ......................: Perseus
 Instruments ...............: all
 Test Statistic ............: -1.28626415205144
 Observation identifiers ...: all
 Model type ................: DiffuseSource
 Model components ..........: "DiffuseMap" * "FileFunction" * "Constant"
 Number of parameters ......: 3
 Number of spatial par's ...: 1
  Normalization ............: 1 [0.001,1000]  (fixed,scale=1,gradient)
 Number of spectral par's .

In [13]:
#cpipe.run_ana_imaging(bkgsubtract='NONE', do_TS=False, do_Res=True, do_Skymap=True, do_SourceDet=False)

=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_EventsSelected.xml
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_SkymapTot.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 31
 nypix .....................: 31
 binsz .....................: 0.1
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.1
 inradius ..................: 0.6
 outradius .................: 0.8
 iterations ................: 3
 threshold .................: 3
 inexclusio

In [14]:
#cpipe.run_ana_spectral(do_Spec=True, do_Butterfly=True, do_Res=True)

--- Computing spectrum: Perseus
=== GApplication ===
 Name ......................: csspec
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Output.xml
 srcname ...................: Perseus
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Bkgcube.fits
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outfile ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Spectrum_Perseus.fits
 method ....................: SLICE
 ebinalg ...................: LOG
 emin

ZeroDivisionError: float division by zero

In [16]:
#cpipe.run_ana_expected_output(obsID=obsID, profile_reso=0.05*u.deg)

=== GApplication ===
 Name ......................: ctmodel
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 incube ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expected_Cluster.xml
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Bkgcube.fits
 caldb .....................: NONE
 irf .......................: NONE
 edisp .....................: no
 outcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expected_Cluster_Counts.fits
 ra ........................: NONE
 d

In [17]:
#cpipe.run_ana_plot(obsID=obsID, profile_log=True, smoothing_FWHM=0.1*u.deg)

----- ObsID to be looked at: ['001', '002', '003']



/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting_irf.py:111: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  c    = sub.imshow(image, extent=[emin,emax,tmin,tmax], aspect=0.5, vmin=1e6, vmax=5e10, norm=LogNorm())
/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting_irf.py:473: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  c    = sub.imshow(image, extent=[emin,emax,tmin,tmax], aspect=0.5,


=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_SelectedEvents001.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Skymap001.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 51
 nypix .....................: 51
 binsz .....................: 0.1
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.1
 inradius ..................: 1
 outradius .................: 2
 iterations ................: 3
 threshold .................: 3
 inexclusio

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:382: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam/Project/anaconda3/envs/cta_phys/lib/python3.8/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)


=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_SelectedEvents002.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Skymap002.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 51
 nypix .....................: 51
 binsz .....................: 0.1
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.1
 inradius ..................: 1
 outradius .................: 2
 iterations ................: 3
 threshold .................: 3
 inexclusio

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:382: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam/Project/anaconda3/envs/cta_phys/lib/python3.8/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)


=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.1
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_SelectedEvents003.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Skymap003.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 51
 nypix .....................: 51
 binsz .....................: 0.1
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.1
 inradius ..................: 1
 outradius .................: 2
 iterations ................: 3
 threshold .................: 3
 inexclusio

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:382: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam/Project/anaconda3/envs/cta_phys/lib/python3.8/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)
/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:382: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam

         This assumes weak noise spatial variarion (w.r.t. smoothing),
         gaussian regime, and uncorrelated pixels.
         This assumes weak noise spatial variarion (w.r.t. smoothing),
         gaussian regime, and uncorrelated pixels.
/Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Spectrum_IC310.fits does not exist, no Spectrum_IC310 plot
/Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Spectrum_Residual.fits does not exist, no Spectrum_Residual plot
